In [1]:
#Installation as per requirement
!pip install fastparquet
!pip install gcsfs
!pip install s3fs

     |████████████████████████████████| 153kB 3.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 17.8MB/s eta 0:00:01
  Created wheel for fastparquet: filename=fastparquet-0.3.2-cp37-cp37m-linux_x86_64.whl size=260871 sha256=22ec60ac41a6971c688544601d967f7006856ed1e6797a75d0c6127655c0cf22
  Stored in directory: /home/jovyan/.cache/pip/wheels/b9/36/13/01416a760ddcab0eb8281ec9c9ffcbed945c9b831647c8b904
  Created wheel for thrift: filename=thrift-0.11.0-cp37-cp37m-linux_x86_64.whl size=390647 sha256=558890364788031e5286fc73a061a6c1c13f437a6558f0f7036c3269b40f9840
  Stored in directory: /home/jovyan/.cache/pip/wheels/be/36/81/0f93ba89a1cb7887c91937948519840a72c0ffdd57cac0ae8f
Successfully built fastparquet thrift
     |████████████████████████████████| 81kB 6.9MB/s eta 0:00:011
     |████████████████████████████████| 102kB 32.2MB/s ta 0:00:01
     |████████████████████████████████| 81kB 30.3MB/s eta 0:00:01
     |████████████████████████████████| 51kB 2.4MB/s eta 0:00:011
   

In [2]:
#Start Dask Client for Dashboard-number of workers=10
from dask.distributed import Client, progress
client = Client(threads_per_worker=1, 
                n_workers=10,
                memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:42149 Dashboard: /user/dask-dask-examples-5e2pbmpy/proxy/8787/status,Cluster Workers: 10 Cores: 10 Memory: 20.00 GB


In [3]:
#Importing libraries and reading data
import dask.dataframe as db
import gcsfs
ff=db.read_parquet('gcs://anaconda-public-data/nyc-taxi/nyc.parquet/part.0.parquet',engine='fastparquet')

In [4]:
ff.head()

,tpep_pickup_datetime,VendorID,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2015-01-01 00:00:00,1,2015-01-01 00:11:26,5,4.00,-73.971436,40.760201,1,N,-73.921181,40.768269,2,13.5,0.5,0.5,0.0,0.0,0.0,14.5
1,2015-01-01 00:00:00,2,2015-01-01 00:00:00,1,1.68,-73.991547,40.750069,1,N,0.000000,0.000000,2,10.0,0.0,0.5,0.0,0.0,0.3,10.8
2,2015-01-01 00:00:00,2,2015-01-01 00:00:00,3,1.56,-74.001320,40.729057,1,N,-74.010208,40.719662,1,7.5,0.5,0.5,0.0,0.0,0.3,8.8
3,2015-01-01 00:00:01,1,2015-01-01 00:03:49,1,0.80,-73.860847,40.757294,1,N,-73.868111,40.752285,2,5.0,0.5,0.5,0.0,0.0,0.0,6.3
4,2015-01-01 00:00:03,2,2015-01-01 00:21:48,2,2.57,-73.969017,40.754269,1,N,-73.994133,40.761600,2,14.5,0.5,0.5,0.0,0.0,0.3,15.8


In [5]:
import dask.dataframe as dd
df = dd.read_csv('s3://nyc-tlc/misc/taxi _zone_lookup.csv',storage_options={'anon': True, 'use_ssl': False})

In [6]:
df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [7]:
#taking out the required variable i.e time as string
ff['pickup_hour']=ff['tpep_pickup_datetime'].astype(str).apply(lambda x: x[11:13],meta='str')

ff['dropoff_hour']=ff['tpep_dropoff_datetime'].astype(str).apply(lambda x: x[11:13],meta='str')

In [8]:
ff.pickup_hour.value_counts().compute()

00    57167
01    54658
02    45702
18    41090
19    39712
15    38708
14    38254
17    38191
03    37813
13    36471
12    35834
22    35727
10    35311
20    35170
16    34825
21    34412
23    32698
09    32219
11    30869
04    26768
08    24440
07    19483
06    16568
05    15379
Name: pickup_hour, dtype: int64

In [10]:
#above table shows the no. of pickups per hour in 24 hours

In [11]:
#similar like above but dropoffs
ff.dropoff_hour.value_counts().compute()

01    56039
00    51182
02    47873
18    40786
19    40772
03    39491
15    38593
14    37462
17    37075
13    36030
16    35892
10    35854
20    35848
22    35498
12    35118
21    34468
23    33211
09    30973
11    30107
04    29740
08    23563
07    19131
05    16558
06    16205
Name: dropoff_hour, dtype: int64

In [12]:
#So therefore we conclude that Peak hours were around 00 i.e. 12 am-1 am
#and off peak hours were around 5am-6am